In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import geopandas as gpd
import pandas as pd
from siuba import *

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium

In [3]:
from update_vars import analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS

In [4]:
import sjoin_stops_to_segments

In [5]:
analysis_date

'2024-10-21'

In [6]:
# (1) Aggregate stop times - by stop_id, find max trips in AM/PM peak
# takes 1 min
max_arrivals_by_stop = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
).pipe(sjoin_stops_to_segments.prep_stop_times).pipe(sjoin_stops_to_segments.stop_times_aggregation_max_by_stop, analysis_date)

In [6]:
# (1) Aggregate stop times - by stop_id, find max trips in AM/PM peak
# takes 1 min
max_arrivals_by_stop_single = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
).pipe(sjoin_stops_to_segments.prep_stop_times).pipe(
    sjoin_stops_to_segments.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=True)

In [7]:
new_hq_multi = max_arrivals_by_stop >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (still multi-route)
new_ms_multi = max_arrivals_by_stop >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (still multi-route)
new_hq_single = max_arrivals_by_stop_single >> filter(_.am_max_trips_hr > 4, _.pm_max_trips_hr > 4) #  new HQ corridor (single-route)
new_ms_single = max_arrivals_by_stop_single >> filter(_.am_max_trips_hr > 3, _.pm_max_trips_hr > 3) #  new major stop precursor (single-route)

## preliminary mapping

* add bus/rail/ferry MTS in orange to help decide

In [8]:
OLD_EXPORT_PATH = f"{GCS_FILE_PATH}export/{rt_dates.DATES['aug2024']}/"

In [9]:
old_stops = gpd.read_parquet(f"{OLD_EXPORT_PATH}ca_hq_transit_stops.parquet")

In [10]:
non_bus = old_stops >> filter(-_.hqta_type.isin(['hq_corridor_bus', 'major_stop_bus']))

In [11]:
feeds = (gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date)
         >> select(_.feed_key, _.schedule_gtfs_dataset_key == _.gtfs_dataset_key, _.name)
         >> filter(_.schedule_gtfs_dataset_key.isin(max_arrivals_by_stop.schedule_gtfs_dataset_key))
        )

In [12]:
stops = gtfs_utils_v2.get_stops(selected_date=analysis_date, operator_feeds=feeds.feed_key,
                       stop_cols=['feed_key', 'stop_id'])

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 500 Query exceeded limit for bytes billed: 5000000000. 12200181760 or higher required.

Location: us-west2
Job ID: d8266571-290f-4b97-bbd3-681202fbdc27

[SQL: SELECT `anon_1`.`feed_key`, `anon_1`.`stop_id`, `anon_1`.`pt_geom` 
FROM (SELECT `anon_2`.`key` AS `key`, `anon_2`.`service_date` AS `service_date`, `anon_2`.`feed_key` AS `feed_key`, `anon_2`.`stop_id` AS `stop_id`, `anon_2`.`feed_timezone` AS `feed_timezone`, `anon_2`.`stop_event_count` AS `stop_event_count`, `anon_2`.`first_stop_arrival_datetime_pacific` AS `first_stop_arrival_datetime_pacific`, `anon_2`.`last_stop_departure_datetime_pacific` AS `last_stop_departure_datetime_pacific`, `anon_2`.`_feed_valid_from` AS `_feed_valid_from`, `anon_2`.`route_type_0` AS `route_type_0`, `anon_2`.`route_type_1` AS `route_type_1`, `anon_2`.`route_type_2` AS `route_type_2`, `anon_2`.`route_type_3` AS `route_type_3`, `anon_2`.`route_type_4` AS `route_type_4`, `anon_2`.`ro

In [ ]:
stops = stops >> inner_join(_, feeds, on='feed_key')

In [ ]:
def add_to_map(test_df, m=None, **kwargs):
    gdf = stops >> inner_join(_, test_df, on=['schedule_gtfs_dataset_key', 'stop_id'])
    m = gdf.explore(m = m, **kwargs)
    return m

In [ ]:
m1 = old_stops.explore(color='blue')
m1 = non_bus.explore(m = m1, color='orange')

In [ ]:
m1 = add_to_map(new_hq_multi, m=m1, color='green')

In [ ]:
m1 = add_to_map(new_hq_single, m=m1, color='red')

In [ ]:
folium.LayerControl().add_to(m1);

In [ ]:
# m1

## major stops

In [ ]:
m = old_stops.explore(color='blue')
m = non_bus.explore(m = m, color='orange')

In [ ]:
m = add_to_map(new_ms_multi, m=m, color='green')

In [ ]:
m = add_to_map(new_ms_single, m=m, color='red')

In [ ]:
folium.LayerControl().add_to(m);

In [ ]:
# m

# full pipeline check

In [ ]:
new_stops = gpd.read_parquet(f"{EXPORT_PATH}ca_hq_transit_stops.parquet")

In [ ]:
new_areas = gpd.read_parquet(f"{EXPORT_PATH}ca_hq_transit_areas.parquet")

In [ ]:
f"{EXPORT_PATH}ca_hq_transit_areas.parquet"

In [ ]:
old_areas = gpd.read_parquet(f"{OLD_EXPORT_PATH}ca_hq_transit_areas.parquet")

In [ ]:
new_stops.info()

In [ ]:
new_stops >> count(_.hqta_type)

In [ ]:
old_stops >> count(_.hqta_type)

In [ ]:
old_stops.info()

In [ ]:
m2 = (old_stops >> filter(_.hqta_type == 'major_stop_bus')).explore(color='blue')
m2 = (new_stops >> filter(_.hqta_type == 'major_stop_bus')).explore(m = m2, color='orange')
folium.LayerControl().add_to(m2);

In [ ]:
# m2

In [ ]:
from calitp_data_analysis.geography_utils import CA_NAD83Albers

In [ ]:
new_dissolved = new_areas.to_crs(CA_NAD83Albers).dissolve(by='hqta_type').reset_index()
new_dissolved['area'] = new_dissolved.geometry.apply(lambda x: x.area)
new_dissolved[['hqta_type', 'area']]

In [ ]:
old_dissolved = old_areas.to_crs(CA_NAD83Albers).dissolve(by='hqta_type').reset_index()
old_dissolved['area'] = old_dissolved.geometry.apply(lambda x: x.area)
old_dissolved[['hqta_type', 'area']]

In [ ]:
old_hq = old_dissolved >> filter(_.hqta_type == 'major_stop_bus')

In [ ]:
new_hq = new_dissolved >> filter(_.hqta_type == "major_stop_bus")

In [ ]:
only_old = old_hq.overlay(new_hq, how='difference')

In [ ]:
only_new = new_hq.overlay(old_hq, how='difference')

In [ ]:
# only_new.explore()

In [ ]:
# only_old.explore()

In [ ]:
import intake

In [ ]:
catalog = intake.open_catalog("*.yml")

In [ ]:
hqta_points = catalog.hqta_points.read().to_crs(PROJECT_CRS)

In [ ]:
hqta_points >> filter(_.stop_id=='62965')

## Counting rail/ferry stops

* Prior refactors have us keeping a row per stop_id x route_id, will keep
* did scan and remove a few ferry stops without bus/rail (Havasu, other Angel Island operator)

In [ ]:
# (new_stops >> filter(_.hqta_type == 'major_stop_ferry')).explore()

In [ ]:
# (old_stops >> filter(_.hqta_type == 'major_stop_ferry')).explore()

In [ ]:
(old_stops >> filter(_.hqta_type == 'major_stop_ferry')) >> count(_.agency_primary)

In [ ]:
(new_stops >> filter(_.hqta_type == 'major_stop_ferry')) >> count(_.agency_primary)

In [ ]:
# old_stops >> filter(_.hqta_type == 'major_stop_ferry', _.agency_primary.str.contains('Golden'))

# new_stops >> filter(_.hqta_type == 'major_stop_ferry', _.agency_primary.str.contains('Golden'))

# new_stops >> filter(_.hqta_type == 'major_stop_rail', _.agency_primary.str.contains('Bay Area'))

# old_stops >> filter(_.hqta_type == 'major_stop_rail', _.agency_primary.str.contains('Bay Area'))

## Finding Amtrak

In [6]:
helpers.import_scheduled_stops?

Signature:
helpers.import_scheduled_stops(
    analysis_date: str,
    filters: tuple = None,
    columns: list = None,
    get_pandas: bool = True,
    crs: str = 'EPSG:3310',
) -> Union[geopandas.geodataframe.GeoDataFrame, dask_geopandas.core.GeoDataFrame]
Docstring: Get scheduled stops
File:      ~/data-analyses/rt_segment_speeds/segment_speed_utils/helpers.py
Type:      function

In [7]:
stops = helpers.import_scheduled_stops(analysis_date=analysis_date)

In [8]:
stops >> head(3)

,feed_key,service_date,feed_timezone,first_stop_arrival_datetime_pacific,last_stop_departure_datetime_pacific,stop_id,stop_key,stop_name,stop_event_count,route_type_0,...,route_type_2,route_type_3,route_type_4,route_type_5,route_type_6,route_type_7,route_type_11,route_type_12,missing_route_type,geometry
0,2cf66736ff937cdf2e49b78da1912e0f,2024-10-16,America/Los_Angeles,2024-10-16 08:20:00,2024-10-16 17:12:00,9892,9f9482c8dc975382f604f166fa5d54fc,Escalon Downtown Park and Ride Lot Main St at ...,6,NaN,...,NaN,6.0,NaN,NaN,None,None,None,None,None,POINT (-87476.215 -24056.085)
1,2cf66736ff937cdf2e49b78da1912e0f,2024-10-16,America/Los_Angeles,NaT,NaT,9894,43e42244a32f3ef511f09181ddb81172,Coley Avenue at 3rd Street - US Post Office,3,NaN,...,NaN,3.0,NaN,NaN,None,None,None,None,None,POINT (-87120.471 -24239.577)
2,2cf66736ff937cdf2e49b78da1912e0f,2024-10-16,America/Los_Angeles,2024-10-16 08:34:00,2024-10-16 16:34:00,9903,8fe6ca9f34e4ab70bf14da88fea1fa32,Kiernan Avenue at McHenry Avenue,3,NaN,...,NaN,3.0,NaN,NaN,None,None,None,None,None,POINT (-87686.256 -33457.962)


In [9]:
all_feeds = (gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date)
         >> select(_.feed_key, _.schedule_gtfs_dataset_key == _.gtfs_dataset_key, _.name))

In [19]:
all_feeds >> filter(_.name.str.contains('Am'))

,feed_key,schedule_gtfs_dataset_key,name
48,1dc776ee08e41c234e41e17374a647ef,36b8fbf12e4adc76b21651462b200860,Amador Schedule
63,8d3e1159c9f7863ff2453b5265faa157,48e137bc977da88970393f629c18432c,Amtrak Schedule


In [ ]:
rt_utils.show_full_df(all_feeds)

In [ ]:
from shared_utils import rt_utils

In [18]:
all_feeds = (gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date='2024-10-21')
         >> select(_.feed_key, _.schedule_gtfs_dataset_key == _.gtfs_dataset_key, _.name))

In [22]:
trips = gtfs_utils_v2.get_trips(selected_date='2024-10-21', operator_feeds=['8d3e1159c9f7863ff2453b5265faa157'])

In [23]:
trips >> distinct(_.route_long_name)

,route_long_name
0,Amtrak Thruway Connecting Service
1,Commuter Rail
2,Lincoln Service
3,Acela
4,Lincoln Service Missouri River Runner
5,City of New Orleans
6,Valley Flyer
7,Amtrak Hartford Line
8,Illinois Zephyr
9,San Joaquins
